## 0. Import Libraries

In [1]:
import re 
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

## 1.Scrape content off lyrics.com

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url ='https://www.lyrics.com'
artist_url = 'https://www.lyrics.com/artist.php?name=Elton-John&aid=4617&o=1'
artist2_url = 'https://www.lyrics.com/artist.php?name=The-Beatles&aid=3644&o=1'


In [3]:
# define variables for code
no_of_songs = 400
x_keywords = 'yeah, submarine, love, shine, you'
x_keywords = [x_keywords]

In [4]:
container = requests.get(artist_url, headers)
container2= requests.get(artist2_url, headers)

In [5]:
container_html = soup(container.text, 'html.parser')
container2_html = soup(container2.text, 'html.parser')


In [6]:
link = container_html.find_all('a', href=True)
link2 = container2_html.find_all('a', href=True)


In [7]:
all_links = [x.get('href') for x in link if r'/lyric' in x.get('href')]
all_links2 = [x.get('href') for x in link2 if r'/lyric' in x.get('href')]


In [8]:
df = pd.DataFrame()
#save all songs in one csv
for a in all_links[:no_of_songs]:
    # open required url
    response = requests.get(url+a, headers)
    # get html text from site
    response_html = soup(response.text, 'html.parser')
    # identify the song text
    response_text=response_html.find_all('pre',attrs={"id":"lyric-body-text"})
    # identify the song title
    response_title = response_html.find('h1',attrs={"class":"lyric-title"}).text
    # loop over song text and extract pure text / get rid of html technical things
    for i in response_text:
        song = i.text
        df = df.append([song],ignore_index=True)


SSLError: HTTPSConnectionPool(host='www.lyrics.com', port=443): Max retries exceeded with url: /lyric/36653891/Elton+John/Bennie+%26+The+Jets?User-Agent=Mozilla%2F5.0+%28Macintosh%3B+Intel+Mac+OS+X+10_10_1%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F39.0.2171.95+Safari%2F537.36 (Caused by SSLError(SSLError("bad handshake: SysCallError(60, 'ETIMEDOUT')")))

In [ ]:
df.to_csv('elton.csv')

In [ ]:
df2 = pd.DataFrame()
#save all songs in one csv
for a in all_links2[:no_of_songs]:
    # open required url
    response2 = requests.get(url+a, headers)
    # get html text from site
    response2_html = soup(response2.text, 'html.parser')
    # identify the song text
    response2_text=response2_html.find_all('pre',attrs={"id":"lyric-body-text"})
    # identify the song title
    response2_title = response2_html.find('h1',attrs={"class":"lyric-title"}).text
    # loop over song text and extract pure text / get rid of html technical things
    for i in response2_text:
        song2 = i.text
        df2 = df2.append([song2],ignore_index=True)

In [ ]:
df2.to_csv('beatles.csv')

In [ ]:
artist_list = [["Elton John"] * no_of_songs + ["The Beatles"] * no_of_songs]
df3 = pd.concat([df, df2], ignore_index=True).set_index(artist_list).rename(columns={0:'songtext'})
df3['str_compare']=df3['songtext'].str.slice(0,20)
df3 = df3.drop_duplicates(subset=['str_compare'], keep='first')
df3 = df3.drop('str_compare', axis =1)
df3

In [ ]:
df3.to_csv('elton_beatles.csv')

In [ ]:
artist_new = df3.index
artist_new


In [ ]:
df3.shape, artist_new.shape

## 2. Apply Bag of Words

In [ ]:
cv = CountVectorizer(stop_words='english')
cv.fit(df3['songtext'])

In [ ]:
x_keywords = pd.DataFrame(x_keywords, columns=['keywords'])
x_keywords_vecs = cv.transform(x_keywords['keywords'])


In [ ]:
df3_vecs = cv.transform(df3['songtext'])
df3_vecs.todense() 

In [ ]:
df3_cv = pd.DataFrame(df3_vecs.todense(), index = artist_new, columns=cv.get_feature_names())
df3_cv

## 3. Apply TfIDf Transformer

In [ ]:
tf = TfidfTransformer()
tf_vecs = tf.fit_transform(df3_vecs)
tf_vecs.shape

In [ ]:
df3_tf = pd.DataFrame(tf_vecs.todense().round(2), index=artist_new, columns=cv.get_feature_names())
df3_tf

In [ ]:
x_keywords_tf_vecs = tf.transform(x_keywords_vecs)
x_keywords_tf = pd.DataFrame(x_keywords_tf_vecs.todense().round(2), columns=cv.get_feature_names())
x_keywords_tf


## 3. Run Naive Bayes Model

In [ ]:
df3_tf.reset_index(inplace=True)
#df3_tf.rename(columns={'level_0':'artist'}, inplace=True).drop('index')
df3_tf

In [ ]:
y = df3_tf['index']
x = df3_tf.drop('index', axis =1)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, random_state=42)

In [ ]:
mn = MultinomialNB(alpha=10000000)

In [ ]:
mn.fit(xtrain, ytrain)

In [ ]:
mn.score(xtest, ytest)

In [ ]:
y_train_pred = mn.predict(xtest)
print('y_train_pred: ',y_train_pred)

In [ ]:
training_score = mn.score(xtrain, ytrain)
training_score